In [1]:
import time

from braket.aws import AwsQuantumJob, AwsSession

from braket.jobs.config import (
    DataSource,
    InputDataConfig,
    InstanceConfig,
    OutputDataConfig,
    S3DataSource,
    StoppingCondition
)

from braket.jobs.metrics_data.definitions import MetricType

In [2]:
# Modify to whatever alias. Only used in naming the job
alias = 'your-alias'

# Pick 'autograd', 'tf', or 'torch'. Autograd is base PennyLane
interface = "autograd"
# SV1, DM1, TN1, or Rigetti. IonQ may take too long
device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
# Specify previous job arn to copy checkpoints from, or None to not copy checkpoints
copy_checkpoints_from_job = None
# copy_checkpoints_from_job = "arn:aws:braket:us-west-2:<your-account-id>:job/<job-name>"

hyperparameters = {
    # We generate a random graph with num_nodes nodes and num_edges edges to run Max Cut on.
    # Number of tasks per iteration = 2 * (num_nodes + num_edges) * p + 1
    # num_nodes is the number of qubits, so each circuit will have runtime exponential in num_nodes
    "num_nodes": "4",
    "num_edges": "5",
    "p": "2",
    "seed": "1967",
    # Maximum number of simultaneous tasks allowed
    "max_parallel": "30",
    # Number of total optimization iterations, including those from previous checkpoint (if any)
    "num_iterations": "3",
    "interface": interface,
}
if copy_checkpoints_from_job:
    hyperparameters['copy_checkpoints_from_job'] = copy_checkpoints_from_job

In [ ]:
# Job data will be sent to this bucket by default,
# unless output_data_config is specified in the next cell.
aws_session = AwsSession()
bucket = aws_session.default_bucket()
print(bucket)

In [4]:
# Other settings to test
# Uncomment or edit to vary
kwargs = {
#     "stopping_condition": StoppingCondition(maxRuntimeInSeconds=5 * 24 * 60 * 60),
#     "output_data_config": OutputDataConfig(s3Path="s3://Your-Bucket-Name"),
#     "instance_config": InstanceConfig(instanceType="ml.m5.large"),
}

In [5]:
# Uncomment to send the Max Cut graph as input data.
# The num_nodes and num_edges hyperparameters will be ignored inside the job,
# since the graph is generated here instead

# import tempfile
# import networkx as nx
# import numpy as np

# def setup_input_stream():
#     graph = nx.gnm_random_graph(
#         int(hyperparameters["num_nodes"]),
#         int(hyperparameters["num_edges"]),
#         seed=int(hyperparameters["seed"])
#     )
#     with tempfile.NamedTemporaryFile() as temp_graph_file:
#         # define the uri for our input data stream
#         stream_s3_uri = aws_session.construct_s3_uri(
#             aws_session.default_bucket(),
#             "input-graph",
#         )
#         # upload something to our input stream location
#         nx.write_adjlist(graph, temp_graph_file.name)
#         aws_session.upload_to_s3(temp_graph_file.name, f"{stream_s3_uri}/input-data.adjlist")
#     return stream_s3_uri

# kwargs["input_data_config"] = [InputDataConfig(
#     channelName="input-graph",
#     dataSource=DataSource(
#         s3DataSource=S3DataSource(
#             s3Uri=setup_input_stream(),
#         ),
#     ),
# )]

In [6]:
region = aws_session.region

# Choose the container based on which one we need.
if interface == 'autograd':
    # Base container does not have PennyLane, so following doesn't work
    # image_uri = f"292282985366.dkr.ecr.{region}.amazonaws.com/base-jobs:1.0-cpu-py37-ubuntu18.04"
    image_uri=f"292282985366.dkr.ecr.{region}.amazonaws.com/tensorflow-jobs:2.4.1-cpu-py37-ubuntu18.04"
elif interface == 'tf':
    image_uri=f"292282985366.dkr.ecr.{region}.amazonaws.com/tensorflow-jobs:2.4.1-cpu-py37-ubuntu18.04"
elif interface == 'torch':
    image_uri=f"292282985366.dkr.ecr.{region}.amazonaws.com/pytorch-jobs:1.8.1-cpu-py37-ubuntu18.04"

In [7]:
start_time = time.time()

job = AwsQuantumJob.create(
    # Note 63-character limit in job name
    job_name="JobTest-"+interface+"-"+alias+"-"+str(int(time.time())),
    image_uri=image_uri,
    # Relative to the current location
    entry_point="source_dir.qaoa_entry_point:start_here",
    device_arn=device_arn,
    source_module="source_dir",
    copy_checkpoints_from_job=copy_checkpoints_from_job,
    # general parameters
    hyperparameters=hyperparameters,
    # Uncomment the following to make this create() call wait for job completion
    # If so, the CloudWatch logs will be printed to the console as well
    # wait_until_complete=True,
    **kwargs
)

print(job.arn)
# Poll the job state asynchronously
while job.state() not in AwsQuantumJob.TERMINAL_STATES:
    print(job.state())
    time.sleep(10)

end_time = time.time()
print(job.state())

arn:aws:braket:us-west-2:491006770141:job/JobTest-autograd-licedric-1631749213
INITIALIZED
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
COMPLETED


In [8]:
print(end_time - start_time)

307.2561218738556


In [9]:
print(job.metadata())

In [10]:
print(job.result())

{'params': [[0.04640119576374078, -0.0006687931947795414], [-0.052651388020882574, -0.03491989905871668]], 'cost': -2.534}


In [11]:
# May need to wait a bit before metrics show up
# If metrics aren't there, run a bit later
time.sleep(120)
print(job.metrics())

{'timestamp': [1631749511.6675212, 1631749508.9265392, 1631749497.1312518, 1631749487.3299847], 'Cost': [-2.534, -2.517, -2.508, -2.495], 'iteration_number': [3.0, 2.0, 1.0, 0.0]}


In [12]:
# Run this cell to print logs from CloudWatch
job.logs()

In [13]:
# Downloads results to current directory
job.download_result()